## PySpark Jupyter Notebook with Python 3 on Windows 11
## Extract the top most frequent used words in reviews for each property listing

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.appName("analytics").getOrCreate()

In [4]:
# define schema for reviews.csv.gz
reviews_schema = StructType([
    StructField('location', StringType(), True),
    StructField('listing_id', StringType(), True),
    #StructField('id',         IntegerType(),True),
    StructField('date',       StringType(),True),
    #StructField('reviewer_id', IntegerType(),True),
    StructField('reviewer_name', StringType(),True),
    StructField('comments', StringType(), True)])

In [5]:
# Read reviews parquet file into data frame reviews_df
cleandata = 'c:/sb/strfacts/cleandata'
reviews_df = spark.read.format("parquet").schema(reviews_schema).load(cleandata)

In [6]:
reviews_df.show(5)

+----------+----------+-------------+--------------------+--------+
|listing_id|      date|reviewer_name|            comments|location|
+----------+----------+-------------+--------------------+--------+
|     13528|2010-01-27|         Mike| Very enjoyable e...|  hawaii|
|    729224|2013-10-22|        Stacy| "We spend an ama...|  hawaii|
|    729224|2013-12-01|      Shannon| "My husband  dau...|  hawaii|
|    729224|2013-12-28|       Nicole| "Very well maint...|  hawaii|
|    729224|2014-06-10|       Ramona| "My family spent...|  hawaii|
+----------+----------+-------------+--------------------+--------+
only showing top 5 rows



In [7]:
# group by list_id, concat all comments
reviews_df.createOrReplaceTempView("reviews")
query = """
    SELECT listing_id, CONCAT_WS(' ', collect_list(comments)) as all_comments, first(location) as area
    FROM reviews
    GROUP BY listing_id
    """
tmpdf = spark.sql(query)
tmpdf.show(2)

+----------+--------------------+------+
|listing_id|        all_comments|  area|
+----------+--------------------+------+
|  10013402| Thank you Jack a...|hawaii|
|  10089978| "We had a wonder...|hawaii|
+----------+--------------------+------+
only showing top 2 rows



In [8]:
# regex remove all nonalphanumeric characters, split by " " and explode words
df1 = tmpdf.select(tmpdf.listing_id,tmpdf.area, F.explode(F.split(F.regexp_replace(tmpdf.all_comments,"[^a-zA-Z0-9 -]","")," ")).alias("words"))

In [9]:
df1.show(10)

+----------+------+-------------+
|listing_id|  area|        words|
+----------+------+-------------+
|  10013402|hawaii|             |
|  10013402|hawaii|        Thank|
|  10013402|hawaii|          you|
|  10013402|hawaii|         Jack|
|  10013402|hawaii|          and|
|  10013402|hawaii|         Nida|
|  10013402|hawaii|          for|
|  10013402|hawaii|        being|
|  10013402|hawaii|         very|
|  10013402|hawaii|accommodating|
+----------+------+-------------+
only showing top 10 rows



In [15]:
# Find the top 5 most frequent used words in the comments
df2 = df1.groupBy(df1.listing_id, df1.area, df1.words).agg(F.count(df1.words).alias("cnt"))

In [16]:
df2.show(5)

+----------+------+-----+---+
|listing_id|  area|words|cnt|
+----------+------+-----+---+
|  10013402|hawaii|     | 37|
|  10013402|hawaii|Thank|  1|
|  10013402|hawaii|  you|  1|
|  10013402|hawaii| Jack| 10|
|  10013402|hawaii|  and| 43|
+----------+------+-----+---+
only showing top 5 rows



In [17]:
windowSpec  = Window.partitionBy(df2.listing_id).orderBy(df2.cnt)
df3 = df2.withColumn("row_number",F.row_number().over(windowSpec))
df3.show(3)

+----------+------+-----+---+----------+
|listing_id|  area|words|cnt|row_number|
+----------+------+-----+---+----------+
|  10013402|hawaii|Thank|  1|         1|
|  10013402|hawaii|  you|  1|         2|
|  10013402|hawaii|being|  1|         3|
+----------+------+-----+---+----------+
only showing top 3 rows



In [18]:
df4 = df3.select("*").where(df3.row_number<=5)
df4.show(5)

+----------+------+-------------+---+----------+
|listing_id|  area|        words|cnt|row_number|
+----------+------+-------------+---+----------+
|  10013402|hawaii|        Thank|  1|         1|
|  10013402|hawaii|          you|  1|         2|
|  10013402|hawaii|        being|  1|         3|
|  10013402|hawaii|accommodating|  1|         4|
|  10013402|hawaii|        funny|  1|         5|
+----------+------+-------------+---+----------+
only showing top 5 rows



In [20]:
df5 = df4.select(df4.listing_id, df4.area, df4.words).groupBy(df4.listing_id, df4.area).agg(F.collect_list(df4.words).alias("frequentw"))

In [21]:
df5.show(5)

+----------+-------+--------------------+
|listing_id|   area|           frequentw|
+----------+-------+--------------------+
|  10013402| hawaii|[Thank, you, bein...|
|  10089978| hawaii|[showerfelt, perf...|
|   1012360| hawaii|[week, breathtaki...|
|  10123910|seattle|[Easily, accommod...|
|   1014157| hawaii|[updates, giant, ...|
+----------+-------+--------------------+
only showing top 5 rows



In [22]:
df6 = df5.select(df5.listing_id, df5.area, F.array_join(df5.frequentw, ",").alias("most_frequent_used_words"))

In [23]:
df6.show(10)

+----------+-------+------------------------+
|listing_id|   area|most_frequent_used_words|
+----------+-------+------------------------+
|  10013402| hawaii|    Thank,you,being,a...|
|  10089978| hawaii|    showerfelt,perfec...|
|   1012360| hawaii|    week,breathtaking...|
|  10123910|seattle|    Easily,accommodat...|
|   1014157| hawaii|    updates,giant,tv,...|
|  10158447| hawaii|    ideally,got,under...|
|  10210560| hawaii|    Many,thanks,swimm...|
|  10214497| hawaii|    Aloha,LOVED,Being...|
|  10217927| hawaii|    AMAZING,sensitive...|
|   1022135|seattle|    just,superb,beaut...|
+----------+-------+------------------------+
only showing top 10 rows



In [24]:
analyticsdata = 'c:/sb/strfacts/analyticsdata'

In [25]:
df6.write.partitionBy("area").mode("append").parquet(analyticsdata)